# Document retrieval from wikipedia data

## 시작

In [151]:
import graphlab
graphlab.set_runtime_config('GRAPHLAB_DEFAULT_NUM_PYLAMBDA_WORKERS', 4)
graphlab.canvas.set_target('ipynb')

# Load some text data - from wikipedia, pages on people

In [152]:
people = graphlab.SFrame('people_wiki.gl/')

Data contains:  link to wikipedia article, name of person, text of article.

In [153]:
people.head()

URI,name,text
<http://dbpedia.org/resource/Digby_Morrell> ...,Digby Morrell,digby morrell born 10october 1979 is a former ...
<http://dbpedia.org/resource/Alfred_J._Lewy> ...,Alfred J. Lewy,alfred j lewy aka sandylewy graduated from ...
<http://dbpedia.org/resource/Harpdog_Brown> ...,Harpdog Brown,harpdog brown is a singerand harmonica player who ...
<http://dbpedia.org/resource/Franz_Rottensteiner> ...,Franz Rottensteiner,franz rottensteiner bornin waidmannsfeld lower ...
<http://dbpedia.org/resource/G-Enka> ...,G-Enka,henry krvits born 30december 1974 in tallinn ...
<http://dbpedia.org/resource/Sam_Henderson> ...,Sam Henderson,sam henderson bornoctober 18 1969 is an ...
<http://dbpedia.org/resource/Aaron_LaCrate> ...,Aaron LaCrate,aaron lacrate is anamerican music producer ...
<http://dbpedia.org/resource/Trevor_Ferguson> ...,Trevor Ferguson,trevor ferguson aka johnfarrow born 11 november ...
<http://dbpedia.org/resource/Grant_Nelson> ...,Grant Nelson,grant nelson born 27april 1971 in london ...
<http://dbpedia.org/resource/Cathy_Caruth> ...,Cathy Caruth,cathy caruth born 1955 isfrank h t rhodes ...


In [154]:
len(people)

59071

# Explore the dataset and checkout the text it contains

## Exploring the entry for president Obama

In [155]:
obama = people[people['name'] == 'Barack Obama']

In [156]:
obama

URI,name,text
<http://dbpedia.org/resource/Barack_Obama> ...,Barack Obama,barack hussein obama iibrk husen bm born august ...


In [157]:
obama['text']

dtype: str
Rows: ?
['barack hussein obama ii brk husen bm born august 4 1961 is the 44th and current president of the united states and the first african american to hold the office born in honolulu hawaii obama is a graduate of columbia university and harvard law school where he served as president of the harvard law review he was a community organizer in chicago before earning his law degree he worked as a civil rights attorney and taught constitutional law at the university of chicago law school from 1992 to 2004 he served three terms representing the 13th district in the illinois senate from 1997 to 2004 running unsuccessfully for the united states house of representatives in 2000in 2004 obama received national attention during his campaign to represent illinois in the united states senate with his victory in the march democratic party primary his keynote address at the democratic national convention in july and his election to the senate in november he began his presidential campa

## Exploring the entry for actor George Clooney

In [158]:
clooney = people[people['name'] == 'George Clooney']
clooney['text']

dtype: str
Rows: ?
['george timothy clooney born may 6 1961 is an american actor writer producer director and activist he has received three golden globe awards for his work as an actor and two academy awards one for acting and the other for producingclooney made his acting debut on television in 1978 and later gained wide recognition in his role as dr doug ross on the longrunning medical drama er from 1994 to 1999 for which he received two emmy award nominations while working on er he began attracting a variety of leading roles in films including the superhero film batman robin 1997 and the crime comedy out of sight 1998 in which he first worked with a director who would become a longtime collaborator steven soderbergh in 1999 clooney took the lead role in three kings a wellreceived war satire set during the gulf warin 2001 clooneys fame widened with the release of his biggest commercial success the heist comedy oceans eleven the first of the film trilogy a remake of the 1960 film wit

# Get the word counts for Obama article

In [159]:
obama['word_count'] = graphlab.text_analytics.count_words(obama['text'])

In [160]:
print obama['word_count']

[{'operations': 1, 'represent': 1, 'office': 2, 'unemployment': 1, 'is': 2, 'doddfrank': 1, 'over': 1, 'unconstitutional': 1, 'domestic': 2, 'named': 1, 'ending': 1, 'ended': 1, 'proposition': 1, 'seats': 1, 'graduate': 1, 'worked': 1, 'before': 1, 'death': 1, '20': 2, 'taxpayer': 1, 'inaugurated': 1, 'obamacare': 1, 'civil': 1, 'mccain': 1, 'to': 14, '4': 1, 'policy': 2, '8': 1, 'has': 4, '2011': 3, '2010': 2, '2013': 1, '2012': 1, 'bin': 1, 'then': 1, 'his': 11, 'march': 1, 'gains': 1, 'cuba': 1, 'californias': 1, '1992': 1, 'new': 1, 'not': 1, 'during': 2, 'years': 1, 'continued': 1, 'presidential': 2, 'husen': 1, 'osama': 1, 'term': 3, 'equality': 1, 'prize': 1, 'lost': 1, 'stimulus': 1, 'january': 3, 'university': 2, 'rights': 1, 'gun': 1, 'republican': 2, 'rodham': 1, 'troop': 1, 'withdrawal': 1, 'involvement': 3, 'response': 3, 'where': 1, 'referred': 1, 'affordable': 1, 'attorney': 1, 'school': 3, 'senate': 3, 'house': 2, 'national': 2, 'creation': 1, 'related': 1, 'hawaii': 1,

## Sort the word counts for the Obama article

### Turning dictonary of word counts into a table

In [161]:
obama_word_count_table = obama[['word_count']].stack('word_count', new_column_name = ['word','count'])

### Sorting the word counts to show most common words at the top

In [162]:
obama_word_count_table.head()

word,count
cuba,1
relations,1
sought,1
combat,1
ending,1
withdrawal,1
state,1
islamic,1
by,1
gains,1


In [163]:
obama_word_count_table.sort('count',ascending=False)

word,count
the,40
in,30
and,21
of,18
to,14
his,11
obama,9
act,8
he,7
a,7


Most common words include uninformative words like "the", "in", "and",...

# Compute TF-IDF for the corpus 

To give more weight to informative words, we weigh them by their TF-IDF scores.

In [164]:
people['word_count'] = graphlab.text_analytics.count_words(people['text'])
people.head()

URI,name,text,word_count
<http://dbpedia.org/resource/Digby_Morrell> ...,Digby Morrell,digby morrell born 10october 1979 is a former ...,"{'selection': 1,'carltons': 1, 'being': ..."
<http://dbpedia.org/resource/Alfred_J._Lewy> ...,Alfred J. Lewy,alfred j lewy aka sandylewy graduated from ...,"{'precise': 1, 'thomas':1, 'closely': 1, ..."
<http://dbpedia.org/resource/Harpdog_Brown> ...,Harpdog Brown,harpdog brown is a singerand harmonica player who ...,"{'just': 1, 'issued': 1,'mainly': 1, 'nominat ..."
<http://dbpedia.org/resource/Franz_Rottensteiner> ...,Franz Rottensteiner,franz rottensteiner bornin waidmannsfeld lower ...,"{'all': 1,'bauforschung': 1, ..."
<http://dbpedia.org/resource/G-Enka> ...,G-Enka,henry krvits born 30december 1974 in tallinn ...,"{'they': 1,'gangstergenka': 1, ..."
<http://dbpedia.org/resource/Sam_Henderson> ...,Sam Henderson,sam henderson bornoctober 18 1969 is an ...,"{'currently': 1, 'less':1, 'being': 1, ..."
<http://dbpedia.org/resource/Aaron_LaCrate> ...,Aaron LaCrate,aaron lacrate is anamerican music producer ...,"{'exclusive': 2,'producer': 1, 'show' ..."
<http://dbpedia.org/resource/Trevor_Ferguson> ...,Trevor Ferguson,trevor ferguson aka johnfarrow born 11 november ...,"{'taxi': 1, 'salon': 1,'gangs': 1, 'being': 1, ..."
<http://dbpedia.org/resource/Grant_Nelson> ...,Grant Nelson,grant nelson born 27april 1971 in london ...,"{'houston': 1, 'frankie':1, 'labels': 1, ..."
<http://dbpedia.org/resource/Cathy_Caruth> ...,Cathy Caruth,cathy caruth born 1955 isfrank h t rhodes ...,"{'phenomenon': 1,'deborash': 1, 'both' ..."


In [165]:
tfidf = graphlab.text_analytics.tf_idf(people['word_count'])

# Earlier versions of GraphLab Create returned an SFrame rather than a single SArray
# This notebook was created using Graphlab Create version 1.7.1
#if graphlab.version <= '1.6.1':
 #   tfidf = tfidf['docs']

type(tfidf)

graphlab.data_structures.sarray.SArray

In [166]:
tfidf[0].keys()

['selection',
 'carltons',
 'being',
 '2005',
 'coach',
 'its',
 'before',
 '21',
 'northern',
 'bullants',
 'to',
 'perth',
 'sydney',
 '2014',
 'has',
 '2011',
 '2013',
 'division',
 'his',
 'rules',
 'assistant',
 'spanned',
 'early',
 'game',
 'five',
 'during',
 'continued',
 '44game',
 'kangaroos',
 'twice',
 'round',
 'the',
 'parade',
 'born',
 'clubs',
 'college',
 'blues',
 'for',
 'falcons',
 'currently',
 'hill',
 'drawn',
 'cause',
 'kicked',
 'exchange',
 'mckernan',
 '19982000',
 'brisbaneafter',
 'losing',
 'essendon',
 'along',
 'teaches',
 'by',
 'box',
 'league',
 'career',
 'of',
 'against',
 'david',
 'melbourne',
 'digby',
 'games',
 'leading',
 'traded',
 'first',
 'goalkicker',
 'morrell',
 'corey',
 'acted',
 'football',
 'carlton',
 'from',
 'district',
 'west',
 'hawks',
 'draft',
 'coburg',
 'forward',
 'australian',
 'was',
 'until',
 'shifted',
 'club',
 'season',
 'vflaffiliate',
 'western',
 'with',
 'former',
 'he',
 '10',
 'october',
 '1979',
 'recruit

In [167]:
people['tfidf'] = tfidf

In [168]:
people

URI,name,text,word_count
<http://dbpedia.org/resource/Digby_Morrell> ...,Digby Morrell,digby morrell born 10october 1979 is a former ...,"{'selection': 1,'carltons': 1, 'being': ..."
<http://dbpedia.org/resource/Alfred_J._Lewy> ...,Alfred J. Lewy,alfred j lewy aka sandylewy graduated from ...,"{'precise': 1, 'thomas':1, 'closely': 1, ..."
<http://dbpedia.org/resource/Harpdog_Brown> ...,Harpdog Brown,harpdog brown is a singerand harmonica player who ...,"{'just': 1, 'issued': 1,'mainly': 1, 'nominat ..."
<http://dbpedia.org/resource/Franz_Rottensteiner> ...,Franz Rottensteiner,franz rottensteiner bornin waidmannsfeld lower ...,"{'all': 1,'bauforschung': 1, ..."
<http://dbpedia.org/resource/G-Enka> ...,G-Enka,henry krvits born 30december 1974 in tallinn ...,"{'they': 1,'gangstergenka': 1, ..."
<http://dbpedia.org/resource/Sam_Henderson> ...,Sam Henderson,sam henderson bornoctober 18 1969 is an ...,"{'currently': 1, 'less':1, 'being': 1, ..."
<http://dbpedia.org/resource/Aaron_LaCrate> ...,Aaron LaCrate,aaron lacrate is anamerican music producer ...,"{'exclusive': 2,'producer': 1, 'show' ..."
<http://dbpedia.org/resource/Trevor_Ferguson> ...,Trevor Ferguson,trevor ferguson aka johnfarrow born 11 november ...,"{'taxi': 1, 'salon': 1,'gangs': 1, 'being': 1, ..."
<http://dbpedia.org/resource/Grant_Nelson> ...,Grant Nelson,grant nelson born 27april 1971 in london ...,"{'houston': 1, 'frankie':1, 'labels': 1, ..."
<http://dbpedia.org/resource/Cathy_Caruth> ...,Cathy Caruth,cathy caruth born 1955 isfrank h t rhodes ...,"{'phenomenon': 1,'deborash': 1, 'both' ..."


## Examine the TF-IDF for the Obama article

In [169]:
obama = people[people['name'] == 'Barack Obama']

In [170]:
obama[['tfidf']].stack('tfidf',new_column_name=['word','tfidf']).sort('tfidf',ascending=False)

word,tfidf
obama,43.2956530721
act,27.678222623
iraq,17.747378588
control,14.8870608452
law,14.7229357618
ordered,14.5333739509
military,13.1159327785
involvement,12.7843852412
response,12.7843852412
democratic,12.4106886973


Words with highest TF-IDF are much more informative.

# Manually compute distances between a few people

Let's manually compare the distances between the articles for a few famous people.  

In [171]:
clinton = people[people['name'] == 'Bill Clinton']

In [172]:
beckham = people[people['name'] == 'David Beckham']

## Is Obama closer to Clinton than to Beckham?

We will use cosine distance, which is given by

(1-cosine_similarity) 

and find that the article about president Obama is closer to the one about former president Clinton than that of footballer David Beckham.

In [173]:
obama['tfidf'][0]

{'13th': 4.9534091674263925,
 '1961': 3.3207419573634955,
 '1992': 2.278351314316948,
 '1996': 2.135691193468776,
 '1997': 2.1298344522079455,
 '20': 4.88376320446593,
 '2000in': 6.250296940830698,
 '2004': 5.071033082507702,
 '2007': 1.4879730697555795,
 '2008': 1.5093391374786154,
 '2009': 4.693309450812809,
 '2010': 3.185667920243947,
 '2011': 5.107041270312875,
 '2012': 1.7938099524877322,
 '2012obama': 10.986495389225194,
 '2013': 1.9545642372230505,
 '4': 2.437803530749586,
 '44th': 7.0744723837970485,
 '63': 5.22130428644035,
 '8': 2.7572509724892824,
 'a': 0.039334291308082026,
 'act': 27.67822262297991,
 'address': 4.8023464982877115,
 'administration': 3.2952952917023315,
 'affordable': 6.134465125305577,
 'afghanistan': 9.4197037997671,
 'african': 3.582216271187926,
 'after': 3.7773337680052257,
 'against': 2.0079609791418744,
 'american': 3.3821333532750204,
 'americans': 4.761936959949835,
 'and': 0.01564802185902329,
 'arms': 5.030658019760364,
 'as': 0.7630171320744707,

In [174]:
graphlab.distances.cosine(obama['tfidf'][0],clinton['tfidf'][0])

0.8339854936884276

In [175]:
graphlab.distances.cosine(obama['tfidf'][0],beckham['tfidf'][0])

0.9791305844747478

In [176]:
graphlab.distances.cosine(obama['tfidf'][0], obama['tfidf'][0])

0.0

# Build a nearest neighbor model for document retrieval

We now create a nearest-neighbors model and apply it to document retrieval.  

In [208]:
knn_model = graphlab.nearest_neighbors.create(people,features=['tfidf'],label='name', distance='cosine')

Starting brute force nearest neighbors model training.

# Applying the nearest-neighbors model for retrieval

## Who is closest to Obama?

In [209]:
knn_model.query(obama)

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.00169288  | 20.794ms     |

| Done         |         | 100         | 171.42ms     |

+--------------+---------+-------------+--------------+

query_label,reference_label,distance,rank
0,Barack Obama,0.0,1
0,Joe Biden,0.703138676734,2
0,Samantha Power,0.742981902328,3
0,Hillary Rodham Clinton,0.758358397887,4
0,Eric Stern (politician),0.770561227601,5


As we can see, president Obama's article is closest to the one about his vice-president Biden, and those of other politicians.  

## Other examples of document retrieval

In [210]:
swift = people[people['name'] == 'Taylor Swift']

In [211]:
knn_model.query(swift)

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.00169288  | 13.48ms      |

| Done         |         | 100         | 132.457ms    |

+--------------+---------+-------------+--------------+

query_label,reference_label,distance,rank
0,Taylor Swift,0.0,1
0,Al Swift,0.699046316493,2
0,Bill Swift,0.712560304479,3
0,Ed Sheeran,0.735540453472,4
0,Carrie Underwood,0.736410347487,5


In [212]:
jolie = people[people['name'] == 'Angelina Jolie']

In [213]:
knn_model.query(jolie)

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.00169288  | 11.891ms     |

| Done         |         | 100         | 140.12ms     |

+--------------+---------+-------------+--------------+

query_label,reference_label,distance,rank
0,Angelina Jolie,0.0,1
0,Brad Pitt,0.730168606132,2
0,Barbara Hershey,0.761752081266,3
0,Keith Jolie,0.770688506797,4
0,Julianne Moore,0.776274899793,5


In [214]:
arnold = people[people['name'] == 'Arnold Schwarzenegger']

In [215]:
knn_model.query(arnold)

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.00169288  | 11.827ms     |

| Done         |         | 100         | 133.495ms    |

+--------------+---------+-------------+--------------+

query_label,reference_label,distance,rank
0,Arnold Schwarzenegger,0.0,1
0,Bonnie Garcia,0.79194972794,2
0,Paul Grant (bodybuilder),0.803523487381,3
0,James Tramel,0.830161679751,4
0,Gray Davis,0.838121872529,5


In [216]:
elton = people[people['name'] == 'Elton John']

In [217]:
elton_word_count_table = elton[['word_count']].stack('word_count', new_column_name = ['word','count'])

# quiz 1


In [218]:
elton_word_count_table.sort('count',ascending=False)

word,count
the,27
in,18
and,15
of,13
a,10
has,9
john,7
he,7
on,6
award,5


# quiz2

In [219]:
elton[['tfidf']].stack('tfidf',new_column_name=['word','tfidf']).sort('tfidf',ascending=False)

word,tfidf
furnish,18.38947184
elton,17.48232027
billboard,17.3036809575
john,13.9393127924
songwriters,11.250406447
tonightcandle,10.9864953892
overallelton,10.9864953892
19702000,10.2933482087
fivedecade,10.2933482087
aids,10.262846934


# quiz3

In [220]:
v_beckham = people[people['name'] == 'Victoria Beckham']

In [221]:
graphlab.distances.cosine(elton['tfidf'][0],v_beckham['tfidf'][0])

0.9567006376655429

# quiz4

In [222]:
paul = people[people['name'] == 'Paul McCartney']

In [223]:
graphlab.distances.cosine(elton['tfidf'][0], paul['tfidf'][0])

0.8250310029221779

# quiz5

In [224]:
simple_model = graphlab.nearest_neighbors.create(people,features=['word_count'],label='name', distance='cosine')

Starting brute force nearest neighbors model training.

In [225]:
v_beckham['word_count']

dtype: dict
Rows: ?
[{'millionin': 1, 'saying': 1, 'cameo': 1, 'beckham': 8, 'show': 1, 'being': 1, 'over': 1, 'ne': 1, 'pops': 1, 'including': 1, 'children': 1, 'decade': 1, 'style': 1, 'group': 1, 'internationally': 1, 'rose': 1, 'had': 3, 'writing': 1, 'late': 1, 'to': 6, 'beckhams': 1, '2014': 1, 'caroline': 1, '2011': 3, '2012': 1, 'real': 1, 'germanys': 1, 'documentaries': 1, 'joint': 1, 'five': 1, 'they': 1, 'judge': 1, '2': 1, 'ticket': 1, 'now': 1, 'four': 2, 'adams': 1, 'businesswoman': 1, 'celebrity': 1, 'married': 1, 'predicted': 1, 'release': 1, 'become': 1, 'shows': 1, 'noted': 1, 'the': 19, 'telstar': 1, 'biannual': 1, 'past': 1, 'born': 1, 'year': 1, 'estimated': 1, 'out': 1, 'allfemale': 1, 'posh': 1, 'for': 1, 'since': 1, 'label': 3, 'won': 1, 'new': 1, 'issue': 1, 'betty': 1, 'who': 1, 'business': 1, 'gathered': 1, '380': 1, 'york': 1, 'belinda': 1, 'by': 1, 'guest': 1, 'on': 1, 'about': 1, 'dubbed': 1, 'of': 6, 'transition': 1, 'british': 1, 'david': 1, 'april': 1, 

In [226]:
v_beckham['tfidf']

dtype: dict
Rows: ?
[{'millionin': 7.728398851203712, 'saying': 4.093853748053105, 'cameo': 5.6393878585077255, 'beckham': 56.922355026538426, 'show': 2.1689013529494012, 'being': 1.7938099524877322, 'over': 1.4878231559557336, 'ne': 4.553555296486015, 'pops': 6.1907048436284535, 'including': 1.2272824458461182, 'children': 2.4252853123921825, 'decade': 3.9816133995123346, 'style': 3.202022153488725, 'group': 1.9855189451548496, 'internationally': 3.60748726159689, 'rose': 3.7120158304513233, 'had': 3.5322852369256745, 'writing': 2.61409678271219, 'late': 2.374355720500003, 'to': 0.2816696260907954, 'beckhams': 9.19473591999714, '2014': 2.2073995783446634, 'caroline': 5.67822769182399, '2011': 5.107041270312875, '2012': 1.7938099524877322, 'real': 3.3707043171893614, 'germanys': 5.936639381975657, 'documentaries': 4.476237048702044, 'joint': 3.769051957528662, 'five': 2.2137301792754096, 'they': 1.8993401178193898, 'judge': 3.466802985108655, '2': 2.2482408126990716, 'ticket': 5.142950

In [227]:
simple_model.query(v_beckham)

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.00169288  | 9.752ms      |

| Done         |         | 100         | 141.838ms    |

+--------------+---------+-------------+--------------+

query_label,reference_label,distance,rank
0,Victoria Beckham,-2.22044604925e-16,1
0,Mary Fitzgerald (artist),0.207307036115,2
0,Adrienne Corri,0.214509782788,3
0,Beverly Jane Fry,0.217466468741,4
0,Raman Mundair,0.217695474992,5


# quiz6

In [228]:
knn_model.query(v_beckham)

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.00169288  | 11.905ms     |

| Done         |         | 100         | 126.719ms    |

+--------------+---------+-------------+--------------+

query_label,reference_label,distance,rank
0,Victoria Beckham,1.11022302463e-16,1
0,David Beckham,0.548169610263,2
0,Stephen Dow Beckham,0.784986706828,3
0,Mel B,0.809585523409,4
0,Caroline Rush,0.819826422919,5
